# BalanseRegnskap og regionsaggregering

## Leser inn nødvendige pakker
- Vi trenger følgende pakker i dette arket:
    - Kostra
    - arrow (for å lese inn parquet filer)
    - Tidyverse (brukes strengt tatt ikke, men fin å ha med om man vil gjøre noe annet med datane)

In [ ]:
library(Kostra)
library("arrow")
library(tidyverse)

In [ ]:
aar <- readline(prompt="Angi årgang:")

## Leser inn nødvendige filer
- **Vi trenger følgende filer for å kjøre BalanseRegnskap**
    - Kasseregnskapet 
        - *Gjør om filen til dataframe* 
        - *Fjerner ekstra kolonne som kommer med i regnskapsfilen. Dette bør ikke komme med. Må finne ut hvorfor å lese inn filen uten denne.*
        - *Endrer kontoklasse fra tall til bokstaver*
        - *Skriver ut de første linjene for å konstrollere at filen blir korrekt. Filen skal inneholde kolonner: periode, region, kontoklasse, art, belop, regnskapsomfang.*

In [ ]:
balanse <- read_parquet(paste('/ssb/stamme03/komakro/pilot_python/arbeidsdata/arbeidsfiler/',aar,'/balanse_basis_',aar,'.parquet', sep = ""))
balanse <- as.data.frame(balanse)
balanse <- select(balanse, -last_col())
head(balanse,2)

In [ ]:
konsolidert <- read_parquet(paste('/ssb/stamme03/komakro/pilot_python/arbeidsdata/arbeidsfiler/',aar,'/kostra0n_0q_basis_',aar,'.parquet', sep = ""))
konsolidert <- as.data.frame(konsolidert)
konsolidert$regnskapsomfang <- gsub('C','B',konsolidert$regnskapsomfang)
konsolidert <- select(konsolidert, -last_col())
head(konsolidert)

- Kapittelhierarki
    - *Gjør om filen til dataframe* 

In [ ]:
kapittelhierarki <- read_parquet(paste('../../../Testdata/arbeidsdata/KostraRegnskap/kapittelhierarki_',aar,'.parquet', sep = ""))
kapittelhierarki <- as.data.frame(kapittelhierarki)
head(kapittelhierarki,2)

- **Vi trenger følgende filer for å kjøre aggregering**
    - Kommune og fylkeskommune hierarki
        - Bydeler ikke med i balanse
    
- Ordner regionshierarkiet
- Hierarkiet lages i "API'er"

In [ ]:
kommunehierarki <- read_parquet(paste('/ssb/stamme03/komakro/pilot_python/arbeidsdata/klass/',aar,'/kommunehierarki.parquet', sep = ""))
kommunehierarki <- as.data.frame(kommunehierarki)
kommunehierarki <- select(kommunehierarki, -last_col())
summary(kommunehierarki)

In [ ]:
fylkeskommunehierarki <- read_parquet(paste('/ssb/stamme03/komakro/pilot_python/arbeidsdata/klass/',aar,'/fylkeskommunehierarki.parquet', sep = ""))
fylkeskommunehierarki <- as.data.frame(fylkeskommunehierarki)
fylkeskommunehierarki <- select(fylkeskommunehierarki, -last_col())
summary(fylkeskommunehierarki)

- **Trenger følgende for å fjerne estimerte kommuner**
    - Kommuner som mangler for 0B og 0D

In [ ]:
manglende <-  read_parquet(paste('/ssb/stamme03/komakro/pilot_python/arbeidsdata/arbeidsfiler/',aar,'/kostra0b_0d_manglende_',aar,'.parquet', sep = ""))
manglende <- as.data.frame(manglende)

## Balanse regnskapsfunksjonen
- Kjører funksjonen BalanseRegnskap fra Kostra biblioteket. Dette er den samme funksjonen som er lagt inn i Kompis.
- Funksjonen må skrives med inputdata i følgende rekkefølge: 
    1. Balanseregnskapet (Kasse og særbedrifter samlet)
    2. Kapittelhierarki

In [ ]:
resultat <- BalanseRegnskap(balanse, kapittelhierarki, omfang = c('A','B','sbedr'))

**Sjekker output**  

Legger på et filter for å kunne kontrollere output.  
Kontrollerer output ved å sammenligne med verdiene i Kompis.

In [ ]:
#summary(resultat)

In [ ]:
#filter(resultat, region == "0301", kapittel == "KG25")

## Konsern
- Filtrerer ut konsern-tall
- Lager KG33, KG34, KG35, KG36
- Aggregerer region
- Fjerner estimerte kommuner

In [ ]:
konsern_start <- filter(resultat, regnskapsomfang == "A")

In [ ]:
ny_var <- resultat %>% 
filter(kapittel %in% c('14','23','33','47') & regnskapsomfang %in% c('B','sbedr')) %>%
pivot_wider(
    names_from = c(regnskapsomfang, kapittel),
    values_from = belop)%>%
mutate (KG33 = B_14 - sbedr_33) %>%
mutate (KG34 = B_23 - sbedr_47) %>%
mutate (KG35 = B_33 - sbedr_14) %>%
mutate (KG36 = B_47 - sbedr_23)

ny_var <- ny_var[c('periode', 'region','KG33','KG34','KG35','KG36')]

ny_var <- ny_var %>% 
pivot_longer(
    cols = starts_with ('KG'), 
    names_to ='kapittel', 
    values_to = 'belop')%>%
add_column(regnskapsomfang = 'A')

In [ ]:
konsern <- rbind(konsern_start, ny_var)

In [ ]:
summary(konsern)

### Aggregerer regnskapet
- Kjører funksjon HierarchyCompute fra kostra biblioteket. 
- Funksjonen aggregerer regnskapet til følgende (så sant regionsfierarkiet er korrekt):
    - EAK (Landet - kommuner)
    - EAKUO (Landet uten Oslo - kommuner)
    - EKAXX (Fylker)
    - EKG (kostragrupper)
    - EAFK (Landet - fylkeskommuner)
    - EAFKUO (Landet uten Oslo - fylkeskommuner)
    - EAFKXX (Landsdeler)
<br>
<br>
- Vi lager kun aggregerte størrelser på konserntall. 

In [ ]:
regionshierarki <- rbind(kommunehierarki, fylkeskommunehierarki)

In [ ]:
# Kjører aggregering
bal_konsern <- HierarchyCompute(data=konsern, valueVar = "belop", 
                      hierarchies=list(region = regionshierarki, regnskapsomfang = "rowFactor",  kapittel = "rowFactor"), 
                      hierarchyVarNames = c(mapsFrom = "from", mapsTo = "to", sign = "sign"), 
                      colVar = "region", rowSelect = unique(konsern[, c("regnskapsomfang", "kapittel")]),
                      inputInOutput = TRUE)

In [ ]:
bal_konsern <- bal_konsern[!bal_konsern$region %in% manglende$region,]

In [ ]:
bal_konsern$periode <- '2021'

In [ ]:
#summary(bal_konsern)

## Kasse
- Filtrerer ut kasse-tall
- Fjerner estimerte kommuner

In [ ]:
bal_kasse <- filter(resultat, regnskapsomfang == "B")

In [ ]:
bal_kasse <- bal_kasse[!bal_kasse$region %in% manglende$region,]

## Konsolidert
- KostraRegnskaps-funksjonen er ikke laget for konsolidert. 
- Konsolidert skal behandles likt som kasse. Mekker derfor til ved å kalle sette regnskapsomfang B på konsolidert og kjøre KostraRegnskap for kun B. 
- Etter kjøringen endres regnskapsomfang tilbake til C. 

In [ ]:
resultat_konsolidert <- BalanseRegnskap(konsolidert, kapittelhierarki, omfang = c('B'))

In [ ]:
resultat_konsolidert$regnskapsomfang <- gsub('B','C',resultat_konsolidert$regnskapsomfang)

## Samler kasse og konsolidert

In [ ]:
bal_kasse_konsolidert <- rbind(bal_kasse, resultat_konsolidert)

In [ ]:
#summary(bal_kasse_konsolidert)

# Ferdige filer
- Skriver det ferdig aggregerte konsernregnskapet til mappe

In [ ]:
lagring_konsern <- paste('/ssb/stamme03/komakro/pilot_python/arbeidsdata/arbeidsfiler/',aar,'/bal_konsern.parquet', sep = "")

In [ ]:
write_parquet(bal_konsern, lagring_konsern)

- Filtrerer ut kasse-tallene fra resultatet av KostraRegnskaps-funksjonen 
- Skriver kasseregnskapet til mappe

In [ ]:
lagring_kasse_konsolidert <- paste('/ssb/stamme03/komakro/pilot_python/arbeidsdata/arbeidsfiler/',aar,'/bal_kasse_konsolidert.parquet', sep = "")

In [ ]:
write_parquet(bal_kasse_konsolidert, lagring_kasse_konsolidert)